In [60]:
import torch
from torch.nn import Linear
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.data import Data, DataLoader
import torch.nn.functional as F
from torch_geometric.utils.convert import from_scipy_sparse_matrix
from torch_geometric.utils.convert import from_networkx
import random
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import pandas as pd
import scipy.special as SS
import scipy.stats as SSA
import copy
import math
from sklearn.model_selection import ParameterGrid
import os
import numpy.linalg as LA
import gzip

from scipy import sparse

# load pickle module
import pickle
import networkx as nx

In [4]:
R0 = 2.1
r = 0.23
save_dir = '../../../../../../Volumes/My Passport/branching/branching_results500/'
file_name1 = save_dir + "NewInf_R0-{}_r-{}.npy.gz" .format(np.round(R0,2),np.round(r,2))
f1 = gzip.GzipFile(file_name1, "r")
x =  np.load(f1)

In [5]:
x.shape

(500, 3142, 60)

In [ ]:
WN = np.loadtxt('W_avg.csv') ### numpy arrary

In [102]:
# Number of nodes (rows of the matrix)
num_nodes = 3142

# Number of features per node (columns of the matrix)
num_features = 60

# Number of classes
num_classes = 5

# Create a random matrix

In [12]:
class MatrixClassifier(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(MatrixClassifier, self).__init__()
        self.conv1 = GCNConv(num_features, 16)
        self.conv2 = GCNConv(16, num_features)
        self.classifier = Linear(num_features, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Use GCN layers to update node features
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)

        # Use global mean pooling to get a representation of the whole graph
        x = global_mean_pool(x, data.batch)

        # Use a linear layer to classify the graph
        x = self.classifier(x)

        return F.log_softmax(x, dim=1)

In [112]:
# Create a graph from mobility matrix
GM = nx.from_numpy_array(WN)
edges = np.array(GM.edges()).transpose()
edge_index = torch.tensor(edges,dtype = torch.int64)

## time series as the features
matrix = torch.from_numpy(x[0])
## r as the categlory/class
y = torch.tensor([1])

data = Data(x=matrix, edge_index=edge_index, y=y)

data.x = data.x.float()
data.y = data.y.long()

# Create a model and an optimizer
model = MatrixClassifier(num_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out, data.y)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

Epoch: 1, Loss: 2.111837387084961
Epoch: 2, Loss: 0.010334550403058529
Epoch: 3, Loss: 0.00023195437097456306
Epoch: 4, Loss: 1.4305012882687151e-05
Epoch: 5, Loss: 1.5497195136049413e-06
Epoch: 6, Loss: 2.3841855067985307e-07
Epoch: 7, Loss: 0.0
Epoch: 8, Loss: 0.0
Epoch: 9, Loss: 0.0
Epoch: 10, Loss: 0.0
Epoch: 11, Loss: 0.0
Epoch: 12, Loss: 0.0
Epoch: 13, Loss: 0.0
Epoch: 14, Loss: 0.0
Epoch: 15, Loss: 0.0
Epoch: 16, Loss: 0.0
Epoch: 17, Loss: 0.0
Epoch: 18, Loss: 0.0
Epoch: 19, Loss: 0.0
Epoch: 20, Loss: 0.0
Epoch: 21, Loss: 0.0
Epoch: 22, Loss: 0.0
Epoch: 23, Loss: 0.0
Epoch: 24, Loss: 0.0
Epoch: 25, Loss: 0.0
Epoch: 26, Loss: 0.0
Epoch: 27, Loss: 0.0
Epoch: 28, Loss: 0.0
Epoch: 29, Loss: 0.0
Epoch: 30, Loss: 0.0
Epoch: 31, Loss: 0.0
Epoch: 32, Loss: 0.0
Epoch: 33, Loss: 0.0
Epoch: 34, Loss: 0.0
Epoch: 35, Loss: 0.0
Epoch: 36, Loss: 0.0
Epoch: 37, Loss: 0.0
Epoch: 38, Loss: 0.0
Epoch: 39, Loss: 0.0
Epoch: 40, Loss: 0.0
Epoch: 41, Loss: 0.0
Epoch: 42, Loss: 0.0
Epoch: 43, Loss: 0.0

In [111]:
model.eval(x[1])

TypeError: eval() takes 1 positional argument but 2 were given